In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import h5py
from functions import process_network_summary

%matplotlib inline
from IPython.display import display, HTML

pd.options.display.float_format = '{:0,.0f}'.format

In [1]:
df = process_network_summary()

df = df[df['county']!="Outside Region"]

NameError: name 'process_network_summary' is not defined

## Truck VMT by County


In [ ]:

df['medium_truck_vmt'] = df['@mveh']*df['length']
df['heavy_truck_vmt'] = df['@hveh']*df['length']
_df = df[['medium_truck_vmt','heavy_truck_vmt','county']].groupby('county').sum()[['medium_truck_vmt','heavy_truck_vmt']]
_df.loc['Total',:] = _df.sum()
_df = _df.reset_index()
_df.rename(columns={'county':'County', 'medium_truck_vmt': 'Medium Trucks', 'heavy_truck_vmt': 'Heavy Trucks'}, inplace=True)
_df


,County,Medium Trucks,Heavy Trucks
0,King,"1,956,074","1,417,574"
1,Kitsap,"145,278","90,379"
2,Outside Region,21,"22,281"
3,Pierce,"549,796","539,816"
4,Snohomish,"551,474","534,242"
5,Total,"3,202,642","2,604,291"


## Congestion by County

In [4]:
df['Medium and Heavy Trucks'] = df['heavy_truck_vmt']+df['medium_truck_vmt']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

,King,Kitsap,Outside Region,Pierce,Snohomish
Light,"618,027","56,276","5,619","235,218","235,357"
Moderate,"131,985",675,NaN,"29,502","24,728"
Heavy,"91,641","1,210",NaN,"8,242","11,592"
Severe,"5,753",44,0,454,449
Total,"847,407","58,205","5,619","273,416","272,126"


In [5]:
_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

,King,Kitsap,Outside Region,Pierce,Snohomish
Light,"353,036","32,912","3,302","136,392","135,562"
Moderate,"97,723",649,NaN,"19,106","18,090"
Heavy,"43,819",642,NaN,"4,436","5,855"
Severe,"2,904",44,0,322,304
Total,"497,483","34,248","3,302","160,256","159,811"


## Truck Miles on FGTS
Freight and Goods Transportation System
- T-1: More than 10 million tons per year
- T-2: 2 4 million to 10 million tons per year

In [6]:
pd.options.display.float_format = '{:0,.0f}'.format
df['Non-Truck'] = df['VMT']-df['Medium and Heavy Trucks']
_df = df[['medium_truck_vmt','heavy_truck_vmt','@fgts','Non-Truck']].melt(
    id_vars='@fgts', var_name='Vehicle Type')
_df = _df.pivot_table(index='Vehicle Type', columns='@fgts', 
                values='value', aggfunc='sum')
# _df = _df.reset_index()
_df = _df.rename_axis(None, axis=1)
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df.rename(index={'heavy_truck_vmt': 'Heavy Trucks',
                 'medium_truck_vmt': 'Medium Trucks'}, inplace=True)
_df[['T-1','T-2','Other Routes']]

,T-1,T-2,Other Routes
Vehicle Type,,,
Non-Truck,"31,090,920","11,378,013","34,012,862"
Heavy Trucks,"2,086,019","172,121","346,151"
Medium Trucks,"1,778,551","486,324","937,767"


In [7]:
pd.options.display.float_format = '{:0,.1%}'.format
_df[['T-1','T-2','Other Routes']]/_df[['T-1','T-2','Other Routes']].sum()

,T-1,T-2,Other Routes
Vehicle Type,,,
Non-Truck,88.9%,94.5%,96.4%
Heavy Trucks,6.0%,1.4%,1.0%
Medium Trucks,5.1%,4.0%,2.7%


## Congestion on FGTS

In [8]:
pd.options.display.float_format = '{:0,.0f}'.format

# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@mveh']+df['@hveh']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

,T-1,T-2,Other Routes
Light,"2,195,091","833,674","1,770,100"
Moderate,"438,267","106,769","212,371"
Heavy,"269,282","93,074","192,064"
Severe,"30,381","35,093","80,674"
Total,"2,933,020","1,068,610","2,255,209"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,78.5%,74.8%,78.0%
Moderate,9.4%,14.9%,10.0%
Heavy,8.5%,9.2%,8.7%
Severe,3.6%,1.0%,3.3%


In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@hveh']+df['@mveh']

_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

,T-1,T-2,Other Routes
Light,"1,249,799","485,045","996,792"
Moderate,"303,472","61,004","129,794"
Heavy,"142,093","51,899","117,574"
Severe,"18,351","23,855","48,575"
Total,"1,713,714","621,802","1,292,735"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,77.1%,72.9%,78.0%
Moderate,10.0%,17.7%,9.8%
Heavy,9.1%,8.3%,8.3%
Severe,3.8%,1.1%,3.8%


## Equity

### Households Within 500' of Heavy Truck Volumes

Total number of households within 500' of T-1 and T-2 routes

- T-1: More than 10 million tons per year

- T-2: 2 4 million to 10 million tons per year

In [12]:
import polars as pl

# Intersect buffer with land use file
df_lu = pl.read_csv(r'..\..\..\..\outputs\landuse\parcels_urbansim.txt', separator=' ').to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))
# Set the coordinate reference system (CRS) to match the land use data
gdf_lu.crs = 'EPSG:2285'

# Buffer the parcels at 500ft
gdf_lu['geometry'] = gdf_lu.buffer(500)

In [13]:

# Intersect this geography  with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
# Do not include connectors since these are abstracted ul3==5; also remove weave links ul3==0 
gdf_network = gdf_network[~gdf_network.ul3.isin([0,5])]
# Truck network links are those that are in FGTS 1 or 2 system
gdf_network = gdf_network[gdf_network['FGTS'].isin([1,2])]

gdf_intersect = gpd.overlay(gdf_network, gdf_lu, how="intersection", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

In [14]:
truck_parcels = gdf_intersect.groupby('parcelid').first()[['hh_p']].reset_index()


# Result should be the network components with some flags for the parcelid
# We can take the parcel information, join with parcel info and group
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),r'../../../../configuration/input_configuration.toml'))

parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])
df = truck_parcels.merge(parcel_geog,left_on='parcelid', right_on='ParcelID')

In [15]:
# Get the total number of households that in equtiy geograhpies
# Comprae the percent of those that are in the buffer versus those that are not
# For the 4 equity groups, perform the calc and add as a table
results_df = pd.DataFrame()
for col, name in {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                        'equity_focus_areas_2023__efa_dis': 'Disability',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
                  }.items():
    _df = df[[col,'hh_p']].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df = pd.concat([results_df,_df])
results_df = results_df.reset_index()
results_df = results_df[results_df['index']>=0]

### Total Households Within 500' of T-1/T-2 Routes by Equity Group

In [16]:
_df = results_df.pivot_table(index='index', columns='equity_group', values='hh_p', aggfunc='sum')
_df.index = _df.index.astype('int').map({0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )
pd.options.display.float_format = '{:0,.0f}'.format
_df.loc['Region',:] = _df.sum(axis=0)
_df.astype('float')

equity_group,Disability,LEP,Older,People of Color,Poverty,Youth
index,,,,,,
Below Regional Average,"96,729","105,618","118,343","75,091","91,169","124,528"
Above Regional Average,"56,908","41,623","53,783","65,912","56,922","45,724"
Higher Share of Equity Population,"33,259","39,655","14,770","45,893","38,805","16,644"
Region,"186,896","186,896","186,896","186,896","186,896","186,896"


In [17]:
pd.options.display.float_format = '{:0,.1%}'.format
_df.drop('Region')/_df.drop('Region').sum()

equity_group,Disability,LEP,Older,People of Color,Poverty,Youth
index,,,,,,
Below Regional Average,51.8%,56.5%,63.3%,40.2%,48.8%,66.6%
Above Regional Average,30.4%,22.3%,28.8%,35.3%,30.5%,24.5%
Higher Share of Equity Population,17.8%,21.2%,7.9%,24.6%,20.8%,8.9%


In [18]:
_df = results_df[["equity_group","hh_p","index"]].pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_buffer = _df.copy()

df_lu_tot = df_lu[['parcelid','hh_p']].merge(parcel_geog,left_on='parcelid', right_on='ParcelID')
df_lu_tot['Region'] = 1
results_df_tot = pd.DataFrame()
for col, name in {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                        'equity_focus_areas_2023__efa_dis': 'Disability',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
                  'Region': 'Region'
                  }.items():
    _df = df_lu_tot[['hh_p',col]].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df_tot = pd.concat([results_df_tot, _df])
results_df_tot = results_df_tot.reset_index()
pd.options.display.float_format = '{:0,.0f}'.format

_df = results_df_tot.pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_tot = _df.copy()
_df = _df_tot.merge(_df_buffer, left_index=True, right_index=True, suffixes=['_tot','_buffer'])

pd.options.display.float_format = '{:0,.1%}'.format
efa_list = ['Below Regional Average', 'Above Regional Average','Higher Share of Equity Population', 'Total Households']
for efa in efa_list:
    _df[efa] = _df[f'{efa}_buffer']/_df[f'{efa}_tot']
_df[efa_list].T


,Disability,LEP,Older,People of Color,Poverty,Youth
Below Regional Average,10.3%,9.7%,13.0%,8.0%,8.8%,13.1%
Above Regional Average,10.8%,11.7%,9.1%,13.2%,12.8%,8.1%
Higher Share of Equity Population,12.5%,13.8%,6.3%,15.7%,15.6%,7.7%
Total Households,10.8%,10.8%,10.8%,10.8%,10.8%,10.8%
